# Maze without Walls

Solution to puzzle from https://fivethirtyeight.com/features/can-you-escape-a-maze-without-walls/


In [1]:
import sys

In [2]:
class Maze():

    # key: prev_dir. Value: dict of square value to new dir
    ACTIONS = {'N': {'S': 'N',
                     'U': 'S',
                     'R': 'E',
                     'L': 'W'},
               'E': {'S': 'E',
                     'U': 'W',
                     'R': 'S',
                     'L': 'N'},
               'S': {'S': 'S',
                     'U': 'N',
                     'R': 'W',
                     'L': 'E'},
               'W': {'S': 'W',
                     'U': 'E',
                     'R': 'N',
                     'L': 'S'},
              }
    
    MOVES = {'N': (0, -1),
             'S': (0, 1),
             'E': (1, 0),
             'W': (-1, 0)
            }
    
    def __init__(self, squares):
        """squares is a list of lists of chars"""
        self.squares = squares
        self.height = len(self.squares)
        self.width = len(self.squares[0])
        
    def next_coord(self, x, y, move):
        """move is a tuple of (x_delta, y_delta)
        
        returns (x, y) of new coord, or None if out of bounds.
        """
        new_x, new_y = x + move[0], y+move[1]
        if new_x < 0 or new_x >= self.width or new_y < 0 or new_y >= self.height:
            return None
        return new_x, new_y
        
        
    def steps_to_exit(self, x, y, prev_dir):
        """Returns tuple of the # of steps to get from here to the exit '!', 
        and a list of the coordinates along the path. 
        
        If we reach "X", returns (-1, None).
        
        x, y are the coords of the current square.
        
        prev_dir is a char ('N', 'S', 'E', or 'W'), giving direction we went to 
        enter this square."""
        args = (x, y, prev_dir)
        if args in self.visited:
            return -1, []
        self.visited.add(args)
        
        cur_sq = self.squares[y][x]
        coord_list = [(x, y)]
        if cur_sq == '!':    # Represent smiley face as "!"
            return 0, coord_list
        if cur_sq == 'X':
            return -1, None
        if cur_sq == '?':
            # Evaluate all directions.
            action_list = ['N', 'S', 'E', 'W']
        else:
            action_list = [self.ACTIONS[prev_dir][cur_sq]]

        best_steps = sys.maxsize
        path = None
        for action in action_list:
            move = self.MOVES[action]
            next_sq = self.next_coord(x, y, move)
            if not next_sq:
                continue
            num_steps, path = self.steps_to_exit(next_sq[0], next_sq[1], action)
            
            if num_steps == -1:
                continue
            
            best_steps = min(best_steps, num_steps+1)
        if path:
            return best_steps, coord_list + path
        return -1, None
    
    def try_entrance(self, x, y, prev_dir):
        self.visited = set()
        num_steps, path = self.steps_to_exit(x, y, prev_dir)
        if num_steps > -1:
            print(f'Found soln! x={x}, y={y}, steps={num_steps}, path={path}')
            return True
        return False
    
    def solve_maze(self):
        """Try all entrances."""
        height = self.height
        width = self.width
        
        solved = False
        
        # Try all entrances from top going S and from bottom going N.
        for x in range(width):
            if self.try_entrance(x, 0, 'S'):
                solved = True
            if self.try_entrance(x, height - 1, 'N'):
                solved = True
            
        # Try all from left going E and from right going W
        for y in range(height):
            if self.try_entrance(0, y, 'E'):
                solved = True
            if self.try_entrance(width - 1, y, 'W'):
                solved = True
                
        return solved            

In [3]:
squares_str = ['LUU?ULXL', 
               'RLRLU!UU',  # Represent smiley face as "!"
               'SLRLULXR',
               'UR?RSL?R',
               'RUURRRSL',
               'S?SLSSLR',
               'RLR?RL?L',
               'LRSRSLRL'
              ]
squares = [[c for c in s] for s in squares_str]

In [4]:
squares

[['L', 'U', 'U', '?', 'U', 'L', 'X', 'L'],
 ['R', 'L', 'R', 'L', 'U', '!', 'U', 'U'],
 ['S', 'L', 'R', 'L', 'U', 'L', 'X', 'R'],
 ['U', 'R', '?', 'R', 'S', 'L', '?', 'R'],
 ['R', 'U', 'U', 'R', 'R', 'R', 'S', 'L'],
 ['S', '?', 'S', 'L', 'S', 'S', 'L', 'R'],
 ['R', 'L', 'R', '?', 'R', 'L', '?', 'L'],
 ['L', 'R', 'S', 'R', 'S', 'L', 'R', 'L']]

In [5]:
maze = Maze(squares)

In [6]:
maze.solve_maze()

False

### So there is no solution. 
However, if we add an additional "!" at x=5, y=4 (0-based coordinates) then there are 3 solutions.

In [7]:
squares_str = ['LUU?ULXL', 
               'RLRLU!UU',
               'SLRLULXR',
               'UR?RSL?R',
               'RUURR!SL',
               'S?SLSSLR',
               'RLR?RL?L',
               'LRSRSLRL'
              ]
squares = [[c for c in s] for s in squares_str]
maze = Maze(squares)

In [8]:
maze.solve_maze()

Found soln! x=3, y=7, steps=7, path=[(3, 7), (4, 7), (5, 7), (5, 6), (4, 6), (4, 5), (4, 4), (5, 4)]
Found soln! x=4, y=7, steps=4, path=[(4, 7), (4, 6), (5, 6), (5, 5), (5, 4)]
Found soln! x=7, y=5, steps=3, path=[(7, 5), (7, 4), (6, 4), (5, 4)]


True